In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import model_selection, metrics   #Additional scklearn functions
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from imblearn.over_sampling import SMOTE
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

xtrain = pd.read_csv("X_train.csv")
ytrain = pd.read_csv("Y_train.csv")
ytrain = ytrain.drop('id', axis = 1)
print(xtrain.shape)
sm = SMOTE(random_state=42)
x_tmp, y_tmp = sm.fit_resample(xtrain.values, ytrain.values)
xtrain = pd.DataFrame(x_tmp, columns = xtrain.columns)
ytrain = pd.DataFrame(y_tmp, columns = ytrain.columns)
xtest = pd.read_csv('X_test.csv')
xtest = xtest.drop("id", axis = 1)
xytrain = pd.concat([ytrain, xtrain], axis = 1).drop("id", axis = 1)

target = 'y'
#IDcol = 'ID'

xytrain.head()
print(xytrain.shape)

(4800, 1001)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(10800, 1001)


In [8]:
predictors = [x for x in xytrain.columns if x not in [target]]

xgb1 = XGBClassifier(
 learning_rate =0.15,
 n_estimators=100,
 max_depth=7,
 min_child_weight=5,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'multi:softmax',
 num_class = 3,
 nthread=4,
 scale_pos_weight=1,
 seed=27)
xgb1.fit(xytrain[predictors], xytrain['y'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.15, max_delta_step=0, max_depth=7,
              min_child_weight=5, missing=None, n_estimators=100, n_jobs=1,
              nthread=4, num_class=3, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=27, silent=None, subsample=0.8, verbosity=1)

In [10]:
ypred = xgb1.predict(xtest)

In [12]:
index = pd.read_csv("sample.csv")
index['y'] = ypred

index.to_csv("test_opt_xgb.csv")